<a href="https://colab.research.google.com/github/oscarastuaq/Predicciones_Ventas_Retail/blob/main/Proyecto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Explora los datos: 
¿Qué necesitas para limpiar estos datos?

¿Hay valores faltantes en el conjunto de datos? 
Es posible que algunas tiendas no informen de todos los datos a causa de problemas técnicos u otros problemas. De ser así, trata con ello apropiadamente.

Aquí hay algunos recursos para que consideres maneras más avanzadas para lidiar con valores faltantes en tu conjunto de datos (revisa los recursos Pandas adicionales opcionales al final de este capítulo para más recursos sobre esto):

1. Datos faltantes, sus tipos y métodos estadísticos para lidiar con ellos

2. 6 formas diferentes de compensar los valores faltantes en un conjunto de datos

3. Valores en un conjunto de datos

Tratamiento de datos fdaltantes: 
1. Supuestos y métodos clave para el análisis aplicados


2. ¿Cuáles son algunos resúmenes estadísticos, información agregada, u otras tendencias que puedes observar en los datos usando Pandas?


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [7]:
filename = '/content/drive/My Drive/CodingDojo/Proyecto_1/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
# con la instruccion info podemos ver en cual columna faltan datos: 
# por ejemplo de los 8523 rows la columna Item_Weight tiene solo 7060 rows con valores. Y 1463 rows con NaN
# tambien la columna Outlet_Size tiene solo 6113 rows con datos. Y  2410 rows con NaN.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [12]:
# verificar cuales valores son NaN o nulos (Null)
df.isna()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,False,False,False,False,False,False,False,False,False,False,False,False
8519,False,False,False,False,False,False,False,False,True,False,False,False
8520,False,False,False,False,False,False,False,False,False,False,False,False
8521,False,False,False,False,False,False,False,False,False,False,False,False


In [13]:
# Verificar si hay datos faltantes por columna
df.isnull().any()



Item_Identifier              False
Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
dtype: bool

In [14]:
# Numero de datos faltantes por columna
df.isnull().sum() # Tenemos dos columnas con datos NaN Item_Weight y Outlet_Size


Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# 1. remove outliers / o valores atipicos.


In [16]:
# 2. remove duplicates
# Metodo para detectar los datos duplicados. 
df.duplicated()  # No tenemos datos duplicados

0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Length: 8523, dtype: bool

In [25]:
# Veamos ahora  para cada tipo de Outlet (Outlet_Type = Supermarket Type1 / Supermarket Type2 / Supermarket Type3 / Grocery Store )
  # cuantos datos son NaN en relacion a la columna Item_Weight  ?

pd.crosstab(index=df.Outlet_Type, columns=df.Item_Weight.isna(), normalize=False)

# Podemos observar que para Grocery Store tenemos 528 datos NaN y que para Supermarket Type 3 tenemos 935 datos NaN
# Adicionalmente podemos observar que para el tipo Supermarket Type 3 No tenemos ningun dato en la columna Item_Weight. 


Item_Weight,False,True
Outlet_Type,,
Grocery Store,555,528
Supermarket Type1,5577,0
Supermarket Type2,928,0
Supermarket Type3,0,935


In [ ]:
# 3. remove unwanted datapoints


In [26]:
# Existe alguna relacion entre los datos faltantes y el tamano de Supermercado?

pd.crosstab(index=df.Outlet_Size, columns=df.Item_Weight.isna(), normalize=False)

# Al parecer en los Supermercados tamano pequeno y medio no estan llegando todos los datos para la columna Item_Weight.

Item_Weight,False,True
Outlet_Size,,
High,932,0
Medium,1858,935
Small,1860,528


In [88]:
# Cual Item tiene mayor  porcentaje de Visualizacion por Nivel donde se ubica dentro del Tipo de Supermercado y su Tamano.

#pandas.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, margins_name='All', dropna=True, normalize=False)

pd.crosstab(index=df.Item_Type,  columns=[  df.Outlet_Type, df.Outlet_Size, df.Outlet_Location_Type , df.Item_Visibility.sum()  ],  normalize='columns')


Outlet_Type           Grocery Store  ... Supermarket Type3
Outlet_Size                   Small  ...            Medium
Outlet_Location_Type         Tier 1  ...            Tier 3
col_3                    563.643281  ...        563.643281
Item_Type                            ...                  
Baking Goods               0.081439  ...          0.073797
Breads                     0.030303  ...          0.033155
Breakfast                  0.018939  ...          0.011765
Canned                     0.071970  ...          0.077005
Dairy                      0.092803  ...          0.071658
Frozen Foods               0.092803  ...          0.095187
Fruits and Vegetables      0.138258  ...          0.149733
Hard Drinks                0.015152  ...          0.024599
Health and Hygiene         0.056818  ...          0.064171
Household                  0.098485  ...          0.105882
Meat                       0.060606  ...          0.059893
Others                     0.032197  ...          0.016043
Seafood                    0.011364  ...          0.007487
Snack Foods                0.142045  ...          0.146524
Soft Drinks                0.049242  ...          0.048128
Starchy Foods              0.007576  ...          0.014973

[16 rows x 7 columns]

In [89]:
#  Con esta agrupacion podemos ver que:
# 1. Los Items categoria Bakking Goods son los que mas porcentaje de area total de visualizacion tienen asignados en la tienda.
# 2. Que los Grocery Store, los cuales son de tamano pequeno y solo tienen un nivel; es donde se le da mayor cantidad de espacion a este item.

df.groupby(['Item_Type','Outlet_Type','Outlet_Size','Outlet_Location_Type'])['Item_Visibility'].sum()

Item_Type      Outlet_Type        Outlet_Size  Outlet_Location_Type
Baking Goods   Grocery Store      Small        Tier 1                  4.918146
               Supermarket Type1  High         Tier 3                  4.342783
                                  Medium       Tier 1                  3.943892
                                  Small        Tier 1                  4.510584
                                               Tier 2                  4.334289
                                                                         ...   
Starchy Foods  Supermarket Type1  Medium       Tier 1                  1.118573
                                  Small        Tier 1                  1.019611
                                               Tier 2                  1.145890
               Supermarket Type2  Medium       Tier 3                  0.934043
               Supermarket Type3  Medium       Tier 3                  0.704562
Name: Item_Visibility, Length: 112, dtype: float64

In [90]:
# Con esta agrupacion podemos ver el monto total de ventas por Item, segun su tipo de Outlet, el tamano del Outlet y el nivel en donde se ubica el Item.
df.groupby(['Item_Type','Outlet_Type','Outlet_Size','Outlet_Location_Type'])['Item_Outlet_Sales'].sum()

Item_Type      Outlet_Type        Outlet_Size  Outlet_Location_Type
Baking Goods   Grocery Store      Small        Tier 1                   14133.6024
               Supermarket Type1  High         Tier 3                  149715.7828
                                  Medium       Tier 1                  139263.3886
                                  Small        Tier 1                  173376.3174
                                               Tier 2                  135005.5976
                                                                          ...     
Starchy Foods  Supermarket Type1  Medium       Tier 1                   42019.9696
                                  Small        Tier 1                   39258.8970
                                               Tier 2                   57945.9056
               Supermarket Type2  Medium       Tier 3                   33110.8998
               Supermarket Type3  Medium       Tier 3                   49170.6616
Name: Item_Outlet_S